In [12]:
import sqlite3
import datetime
import pandas as pd
import requests
from retrying import retry


conn = sqlite3.connect("../db/bn.db")
cur = conn.cursor()
print("数据库打开成功")

数据库打开成功


In [3]:

# 币安API的基本URL
base_url = "https://fapi.binance.com"


@retry
def get_symbol_usdt_id():
    # 发送请求以获取所有交易对
    session = requests.Session()
    response = session.get(f"{base_url}/fapi/v1/exchangeInfo")
    if response.status_code == 200:
        exchange_info = response.json()

        # 遍历所有交易对，并筛选出USDT作为计价货币的永续合约
        usdt_perpetual_pairs = [
            pair["symbol"]
            for pair in exchange_info["symbols"]
            if pair["quoteAsset"] == "USDT" and pair["contractType"] == "PERPETUAL"
        ]
    else:
        print(f"请求失败，状态码：{response.status_code}")
    return usdt_perpetual_pairs

In [4]:


@retry
def get_kline(start_time, end_time, symbol, interval="1m"):
    """
    # 获取合约交易对
    symbol = 'BTCUSDT'

    # 设置时间间隔（1分钟K线）
    interval = '1m'

    # 获取当前时间和30天前的时间
    end_time = int(datetime.datetime.now().timestamp() * 1000)
    start_time = end_time - (30* 24 * 60 * 60 * 1000)  # 30天前的时间
    """

    session = requests.Session()
    # 存储K线数据的列表
    kline_data_list = []

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "limit": 100,  # 最大限制
    }

    response = session.get(f"{base_url}/fapi/v1/klines", params=params)
    if response.status_code == 200:
        kline_data_list = response.json()
    else:
        print(f"获取K线数据失败，状态码：{response.status_code}")

    # 将K线数据转换为DataFrame
    kline_df = pd.DataFrame(
        kline_data_list,
        columns=[
            "timestamp",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "close_time",
            "quote_asset_volume",
            "number_of_trades",
            "taker_buy_base_asset_volume",
            "taker_buy_quote_asset_volume",
            "ignore",
        ],
    )
    kline_df["timestamp"] = pd.to_datetime(kline_df["timestamp"], unit="ms")
    kline_df["timestamp"] = kline_df["timestamp"].astype(str)
    kline_df["symbol"] = symbol
    kline_df["timestamp_symbol"] = kline_df["timestamp"] + symbol
    return kline_df


end = int(datetime.datetime.now().timestamp() * 1000)
start = end - (24 * 60 * 60 * 1000 * 1)
klineData = get_kline(start, end, "BTCUSDT")

In [5]:
def klines_df_sql(_data):  # 标记k线交易数据导入数据库
    _data['symbol'] = "BTCUSDT"
    _data['timestamp_symbol'] = _data['timestamp']+_data['symbol']
    cols = (
        "timestamp_symbol",
        "timestamp",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "close_time",
        "quote_asset_volume",
        "number_of_trades",
        "taker_buy_base_asset_volume",
        "taker_buy_quote_asset_volume",
        "ignore",
    )
    cur.execute('''
        CREATE TABLE IF NOT EXISTS klines(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp_symbol TEXT,
        timestamp INTEGER,
        symbol TEXT,
        open TEXT,
        high TEXT,
        low TEXT,
        close TEXT,
        volume TEXT,
        close_time INTEGER,
        quote_asset_volume TEXT,
        number_of_trades INTEGER,
        taker_buy_base_asset_volume TEXT,
        taker_buy_quote_asset_volume TEXT,
        ignore TEXT)
    ''')
    conn.commit()
    for x in _data.index:
        data_list = str(tuple([_data.loc[x, _col] for _col in cols]))
        cur.execute(
            "INSERT OR IGNORE INTO klines " + str(cols) + " VALUES " + data_list
        )
    conn.commit()


klines_df_sql(klineData)

In [6]:
# 拿db数据

klineDataDb = pd.read_sql_query('SELECT * FROM klines',conn)
klineDataDb


,id,timestamp_symbol,timestamp,symbol,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1,2023-11-12 02:50:00BTCUSDT,2023-11-12 02:50:00,BTCUSDT,36950.60,36950.60,36935.90,36942.60,60.270,1699757459999,2226468.89160,828,17.078,630903.28060,0
1,2,2023-11-12 02:51:00BTCUSDT,2023-11-12 02:51:00,BTCUSDT,36942.60,36943.60,36926,36926,33.041,1699757519999,1220310.11160,761,6.001,221647.68650,0
2,3,2023-11-12 02:52:00BTCUSDT,2023-11-12 02:52:00,BTCUSDT,36926,36939.80,36926,36931.40,25.095,1699757579999,926797.03100,601,16.199,598252.85530,0
3,4,2023-11-12 02:53:00BTCUSDT,2023-11-12 02:53:00,BTCUSDT,36931.40,36931.40,36902.60,36902.60,54.981,1699757639999,2029556.53600,1173,6.407,236526.60520,0
4,5,2023-11-12 02:54:00BTCUSDT,2023-11-12 02:54:00,BTCUSDT,36902.60,36903,36893.20,36898.10,70.311,1699757699999,2594451.65870,1274,31.307,1155201.15520,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,2023-11-13 11:32:00BTCUSDT,2023-11-13 11:32:00,BTCUSDT,36833.90,36853.60,36833.40,36848.70,237.503,1699875179999,8750761.65910,3204,137.330,5059869.47200,0
196,197,2023-11-13 11:33:00BTCUSDT,2023-11-13 11:33:00,BTCUSDT,36848.70,36861.90,36845.20,36845.20,168.223,1699875239999,6199469.61630,1845,116.901,4308082.90990,0
197,198,2023-11-13 11:34:00BTCUSDT,2023-11-13 11:34:00,BTCUSDT,36845.20,36861.70,36841.50,36849.40,106.913,1699875299999,3939682.38040,1415,56.012,2063988.37270,0
198,199,2023-11-13 11:35:00BTCUSDT,2023-11-13 11:35:00,BTCUSDT,36849.50,36861.90,36836.50,36856.50,105.740,1699875359999,3896460.01960,1761,55.117,2031042.93580,0


In [11]:
ma20 = klineDataDb['close'].rolling(window=20).mean()
ma10 = klineDataDb['close'].rolling(window=10).mean()
klineDataDb['ma10'] = ma10
klineDataDb['ma20'] = ma20

In [8]:
class BacktestEngine:
    def __init__(self, fee_pro=0.01):
        # 初始化交易引擎
        self.cash = 0
        self.positions = 0  # 投资组合，包括现金
        self.trade_fee = 0  # 累计交易成本
        self.profit = 0  # 累计盈亏
        self.fee_pro = fee_pro  # 交易手续费率+滑点设置比例
        self.trades_df = pd.DataFrame(
            columns=['datetime', 'cash', 'positions', 'vol', 'price', 'trade_fee', 'profit'])  # 成交记录
        self.data = None  # 历史价格数据

    def load_data(self, data):
        # 载入历史价格数据
        self.data = data

    def execute_order(self, datetime, vol, price):
        cost = price * vol
        self.trade_fee = self.trade_fee + abs(cost) * self.fee_pro  # 更新累计手续费
        self.cash = self.cash - cost - abs(cost) * self.fee_pro  # 更新现金
        self.positions = self.positions + vol
        self.profit = self.cash + self.positions * price

        __df = pd.DataFrame([{'datetime': datetime, 'cash': self.cash, 'positions': self.positions, 'vol': vol,
                              'price': price, 'trade_fee': self.trade_fee, 'profit': self.profit}])
        self.trades_df = pd.concat([self.trades_df, __df])  # 记录成交


In [13]:

bt_engine = BacktestEngine()

bt_engine.load_data(klineDataDb)

for i, row in klineDataDb.iterrows():
    if(i <= 18): continue
    datetime = row["timestamp"]
    price = row["close"]
    if float(row["ma10"]) > float(row["ma20"]) and bt_engine.positions == 0:
        vol = 1
        action = "buy"
        bt_engine.execute_order(datetime, float(vol), float(price))
        # print(f"Datetime: {datetime}, Action: {action}, Volume: {vol}, Price: {price}")
    elif float(row["ma10"]) < float(row["ma20"]) and bt_engine.positions == 1:
        vol = -1
        action = "sell"
        bt_engine.execute_order(datetime, float(vol), float(price))
        # print(f"Datetime: {datetime}, Action: {action}, Volume: {vol}, Price: {price}")


Datetime: 2023-11-12 03:13:00, Action: buy, Volume: 1, Price: 36952.90
Datetime: 2023-11-12 03:26:00, Action: sell, Volume: -1, Price: 36938.90
Datetime: 2023-11-12 03:29:00, Action: buy, Volume: 1, Price: 36941.40
Datetime: 2023-11-12 04:19:00, Action: sell, Volume: -1, Price: 36990.10
Datetime: 2023-11-13 10:00:00, Action: buy, Volume: 1, Price: 37015.30
Datetime: 2023-11-13 10:11:00, Action: sell, Volume: -1, Price: 36985.90
Datetime: 2023-11-13 10:39:00, Action: buy, Volume: 1, Price: 36969.80
Datetime: 2023-11-13 11:01:00, Action: sell, Volume: -1, Price: 36990.20
Datetime: 2023-11-13 11:28:00, Action: buy, Volume: 1, Price: 36880.90
Datetime: 2023-11-13 11:30:00, Action: sell, Volume: -1, Price: 36852.40


/var/folders/v4/dr2f8rqs6dgfs68xcxl_mwrr0000gn/T/ipykernel_76218/1177667930.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.trades_df = pd.concat([self.trades_df, __df])  # 记录成交


In [14]:
# 打印交易记录
print(bt_engine.trades_df)

              datetime       cash  positions  vol    price  trade_fee  \
0  2023-11-12 03:13:00 -37322.429        1.0  1.0  36952.9    369.529   
0  2023-11-12 03:26:00   -752.918        0.0 -1.0  36938.9    738.918   
0  2023-11-12 03:29:00 -38063.732        1.0  1.0  36941.4   1108.332   
0  2023-11-12 04:19:00  -1443.533        0.0 -1.0  36990.1   1478.233   
0  2023-11-13 10:00:00 -38828.986        1.0  1.0  37015.3   1848.386   
0  2023-11-13 10:11:00  -2212.945        0.0 -1.0  36985.9   2218.245   
0  2023-11-13 10:39:00 -39552.443        1.0  1.0  36969.8   2587.943   
0  2023-11-13 11:01:00  -2932.145        0.0 -1.0  36990.2   2957.845   
0  2023-11-13 11:28:00 -40181.854        1.0  1.0  36880.9   3326.654   
0  2023-11-13 11:30:00  -3697.978        0.0 -1.0  36852.4   3695.178   

     profit  
0  -369.529  
0  -752.918  
0 -1122.332  
0 -1443.533  
0 -1813.686  
0 -2212.945  
0 -2582.643  
0 -2932.145  
0 -3300.954  
0 -3697.978  
